# Cryptocurrency: Analysis and Forecasting

## 7. References and Resources
### 7.1 References
[1]: Use our data. (n.d.). Investing Support. https://www.investing-support.com/hc/en-us/articles/360002357417

### 7.2 Resources used